In [6]:
# BRCF.2.CleanerData.ipynb 
# Capture what Caraciolo's implementation used from BX-Books and BX-Book-Ratings
# and save them in true comma-seaparate files: MC.Books.csv and MC.ratings.csv.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def loadDataset(path=""):
	
	#Recover the titles of the books
	fbooks = open(path+"MC.Books.csv", 'w')
	books = {}
	for line in open(path+"BX-Books.csv"):
		line = line.replace('"', "")
		line = line.replace("\\","")
		line = line.replace("\n","")        
		(id,title) = line.split(";") [0:2]
		books[id] = title
		fbooks.write(id + ',' + title + '\n')       
	fbooks.close()    
	
	#Load the data
	prefs = {}
	countVal = 0
	countKey = 0
	fratings = open(path+"MC.Ratings.csv", 'w')
	for line in open(path+"BX-Book-Ratings.csv"):
		line = line.replace('"', "")
		line = line.replace("\\","")
		line = line.replace("\n","")        
		(user,bookid,rating) = line.split(";")
		try:
			if float(rating) > 0.0:
				fratings.write(user + ',' + bookid + ',' + rating + '\n')                                
				prefs.setdefault(user,{})
				prefs[user][books[bookid]] = float(rating)
		except ValueError:
			countVal+=1
#			print "value error found! " + user + bookid + rating
		except KeyError:
			countKey +=1
#			print "key error found! " + user + " " + bookid
	print countVal
	print countKey
	fratings.close()    
	
	return 0


In [3]:
loadDataset('data/')

1
49818


0

In [3]:
# line count
# BX-Books.csv = 271,380
# MC.Books.csv = 271,380
# BX-Book-Ratings.csv = 1,149,781
#      MC.Ratings.csv =   433,671
#
# We also need to eliminate one line from MC.Ratings.csv with editor:
# Line "130499,,.0330486187,6" as there are more than 3 fields